# Import all necessary libraries

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import random
# Dependencies and Setup
# SQL Alchemy
from sqlalchemy import create_engine

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, PrimaryKeyConstraint

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
import sqlite3

# Load and clean data set
## EV charging station by location

In [3]:
stations_df = pd.read_csv('../data/alt_fuel_stations.csv')
stations_df.head()

,Station Name,ZIP,Latitude,Longitude
0,LADWP - Truesdale Center,91352,34.248319,-118.387971
1,LADWP - West LA District Office,90024,34.052542,-118.448504
2,Los Angeles Convention Center,90015,34.040539,-118.271387
3,LADWP - John Ferraro Building,90012,34.059133,-118.248589
4,LADWP - Haynes Power Plant,90803,33.759802,-118.096665


In [4]:
stations_df.columns

Index(['Station Name', 'ZIP', 'Latitude', 'Longitude'], dtype='object')

In [5]:
# keep only wanted columns & Reorder - make ID column first
stations_df = stations_df[['ID','Station Name','Street Address','City','State','ZIP','Station Phone','Status Code',
                           'Groups With Access Code','Access Days Time','Geocode Status',
       'Latitude', 'Longitude','Open Date','EV Connector Types','Access Code', 'Facility Type', 'EV Pricing']]

KeyError: "['ID', 'Street Address', 'City', 'State', 'Station Phone', 'Status Code', 'Groups With Access Code', 'Access Days Time', 'Geocode Status', 'Open Date', 'EV Connector Types', 'Access Code', 'Facility Type', 'EV Pricing'] not in index"

In [ ]:
# Rename column names
stations_df.rename(columns={'ID': 'id', 'Station Name':'station_name', 'Street Address': 'street_address',
                            'City': 'city', 'State': 'state','ZIP': 'zip','Station Phone': 'station_phone',
                            'Status Code': 'status_code', 'Groups With Access Code': 'groups_access_code', 
                            'Access Days Time': 'access_days_time', 'Geocode Status': 'geocode status',
                            'Latitude': 'latitude', 'Longitude': 'longitude','Open Date':'open_date','EV Connector Types':'ev_connector_types',
                            'Access Code':'access_code', 'Facility Type':'facility_type' , 'EV Pricing':'ev_pricing'
                            }, inplace=True)

In [6]:
stations_df.head()

,Station Name,ZIP,Latitude,Longitude
0,LADWP - Truesdale Center,91352,34.248319,-118.387971
1,LADWP - West LA District Office,90024,34.052542,-118.448504
2,Los Angeles Convention Center,90015,34.040539,-118.271387
3,LADWP - John Ferraro Building,90012,34.059133,-118.248589
4,LADWP - Haynes Power Plant,90803,33.759802,-118.096665


In [7]:
print(stations_df.columns)

Index(['Station Name', 'ZIP', 'Latitude', 'Longitude'], dtype='object')


In [8]:
stations_df.fillna(0)

,Station Name,ZIP,Latitude,Longitude
0,LADWP - Truesdale Center,91352,34.248319,-118.387971
1,LADWP - West LA District Office,90024,34.052542,-118.448504
2,Los Angeles Convention Center,90015,34.040539,-118.271387
3,LADWP - John Ferraro Building,90012,34.059133,-118.248589
4,LADWP - Haynes Power Plant,90803,33.759802,-118.096665
...,...,...,...,...
50265,MARIPOSAON3RD SPACE 13,90004,34.069280,-118.299479
50266,MARIPOSAON3RD SPACE 14,90004,34.069280,-118.299538
50267,MARIPOSAON3RD SPACE 11,90004,34.069204,-118.299361
50268,TOWN OF NATICK KMS #2,1760,42.291105,-71.384538


In [9]:
stations_df.size

201080

In [10]:
stations_df.describe()

,Latitude,Longitude
count,50270.000000,50270.000000
mean,37.661903,-98.485662
std,5.012525,19.776861
min,18.334138,-159.788556
25%,34.023393,-118.333923
50%,37.927566,-95.337205
75%,41.122613,-80.142582
max,64.852466,0.000000


## use web scrape to see how average temperatures in different regions have increased over time

In [11]:
url = 'https://science2017.globalchange.gov/chapter/6/'
table = pd.read_html(url)
temp_df = table[2]
temp_df

,NCA Region,Change in Annual Average Temperature,Change in Annual Average Maximum Temperature,Change in Annual Average Minimum Temperature
0,Contiguous U.S.,1.23°F,1.06°F,1.41°F
1,Northeast,1.43°F,1.16°F,1.70°F
2,Southeast,0.46°F,0.16°F,0.76°F
3,Midwest,1.26°F,0.77°F,1.75°F
4,Great Plains North,1.69°F,1.66°F,1.72°F
5,Great Plains South,0.76°F,0.56°F,0.96°F
6,Southwest,1.61°F,1.61°F,1.61°F
7,Northwest,1.54°F,1.52°F,1.56°F
8,Alaska,1.67°F,1.43°F,1.91°F
9,Hawaii,1.26°F,1.01°F,1.49°F


In [12]:
## convert tabulated data to html if needed

temp_table = temp_df.to_html()

In [20]:
## load states CO2 emission

carbon_dioxide = "../data/StatesCO2_emission_2.csv"
carbon_dioxide_df = pd.read_csv(carbon_dioxide)

# US_co2 = carbon_dioxide_df.loc[carbon_dioxide_df["Entity"] == "United States"]
# US_co2_df = US_co2.tail(30)
# US_co2_df
carbon_dioxide_df

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,Percent,Absolute
0,Alabama,142.9,134.2,139.2,140.3,142.6,144.3,146.6,148.2,140.4,...,130.3,123.6,121.1,123.4,120.0,114.8,109.5,113.3,-0.21,-29.6
1,Alaska,44.4,43.5,43.7,43.7,46.9,48.2,45.9,44.2,39.6,...,37.8,36.8,34.8,34.5,35.5,34.0,34.4,35.1,-0.21,-9.3
2,Arizona,86.4,88.7,88.1,90.1,97.1,97.2,100.3,102.3,102.6,...,97.2,95.0,99.0,97.1,94.7,90.7,90.3,93.9,0.09,7.5
3,Arkansas,63.8,63.0,61.5,62.3,62.7,60.4,62.2,63.5,64.4,...,67.5,66.3,68.7,69.2,59.4,62.5,64.5,71.1,0.12,7.3
4,California,376.8,381.3,379.7,368.7,386.6,383.8,392.2,397.2,381.2,...,340.9,346.6,348.2,344.0,351.0,352.3,355.3,356.6,-0.05,-20.2
5,Colorado,85.0,93.1,91.3,90.8,93.7,95.9,96.9,99.5,97.6,...,92.9,91.6,92.6,93.4,91.9,89.0,89.3,90.6,0.07,5.6
6,Connecticut,43.0,41.7,40.1,42.8,44.6,44.1,41.1,40.2,37.7,...,35.1,34.4,34.8,35.2,36.5,34.3,34.1,37.6,-0.13,-5.4
7,Delaware,16.0,15.5,15.3,15.8,16.1,16.8,15.8,16.6,15.7,...,12.6,13.7,13.3,13.0,13.1,13.7,13.0,13.3,-0.17,-2.7
8,District of Columbia,4.3,4.1,4.2,3.9,4.0,3.9,3.2,3.4,3.1,...,3.1,2.7,2.8,3.0,3.0,2.8,2.6,2.9,-0.33,-1.4
9,Florida,240.2,239.0,242.4,246.7,258.1,261.4,259.7,257.4,239.0,...,233.0,227.6,227.8,234.1,238.6,240.2,239.5,243.2,0.01,3.0


In [21]:
states_co2 = "../data/StatesCO2_emission_2.csv"
states_co2 = pd.read_csv(states_co2)



states_co2_df = states_co2
states_co2_df


,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,Percent,Absolute
0,Alabama,142.9,134.2,139.2,140.3,142.6,144.3,146.6,148.2,140.4,...,130.3,123.6,121.1,123.4,120.0,114.8,109.5,113.3,-0.21,-29.6
1,Alaska,44.4,43.5,43.7,43.7,46.9,48.2,45.9,44.2,39.6,...,37.8,36.8,34.8,34.5,35.5,34.0,34.4,35.1,-0.21,-9.3
2,Arizona,86.4,88.7,88.1,90.1,97.1,97.2,100.3,102.3,102.6,...,97.2,95.0,99.0,97.1,94.7,90.7,90.3,93.9,0.09,7.5
3,Arkansas,63.8,63.0,61.5,62.3,62.7,60.4,62.2,63.5,64.4,...,67.5,66.3,68.7,69.2,59.4,62.5,64.5,71.1,0.12,7.3
4,California,376.8,381.3,379.7,368.7,386.6,383.8,392.2,397.2,381.2,...,340.9,346.6,348.2,344.0,351.0,352.3,355.3,356.6,-0.05,-20.2
5,Colorado,85.0,93.1,91.3,90.8,93.7,95.9,96.9,99.5,97.6,...,92.9,91.6,92.6,93.4,91.9,89.0,89.3,90.6,0.07,5.6
6,Connecticut,43.0,41.7,40.1,42.8,44.6,44.1,41.1,40.2,37.7,...,35.1,34.4,34.8,35.2,36.5,34.3,34.1,37.6,-0.13,-5.4
7,Delaware,16.0,15.5,15.3,15.8,16.1,16.8,15.8,16.6,15.7,...,12.6,13.7,13.3,13.0,13.1,13.7,13.0,13.3,-0.17,-2.7
8,District of Columbia,4.3,4.1,4.2,3.9,4.0,3.9,3.2,3.4,3.1,...,3.1,2.7,2.8,3.0,3.0,2.8,2.6,2.9,-0.33,-1.4
9,Florida,240.2,239.0,242.4,246.7,258.1,261.4,259.7,257.4,239.0,...,233.0,227.6,227.8,234.1,238.6,240.2,239.5,243.2,0.01,3.0


In [22]:
states_co2_df.fillna(0)

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,Percent,Absolute
0,Alabama,142.9,134.2,139.2,140.3,142.6,144.3,146.6,148.2,140.4,...,130.3,123.6,121.1,123.4,120.0,114.8,109.5,113.3,-0.21,-29.6
1,Alaska,44.4,43.5,43.7,43.7,46.9,48.2,45.9,44.2,39.6,...,37.8,36.8,34.8,34.5,35.5,34.0,34.4,35.1,-0.21,-9.3
2,Arizona,86.4,88.7,88.1,90.1,97.1,97.2,100.3,102.3,102.6,...,97.2,95.0,99.0,97.1,94.7,90.7,90.3,93.9,0.09,7.5
3,Arkansas,63.8,63.0,61.5,62.3,62.7,60.4,62.2,63.5,64.4,...,67.5,66.3,68.7,69.2,59.4,62.5,64.5,71.1,0.12,7.3
4,California,376.8,381.3,379.7,368.7,386.6,383.8,392.2,397.2,381.2,...,340.9,346.6,348.2,344.0,351.0,352.3,355.3,356.6,-0.05,-20.2
5,Colorado,85.0,93.1,91.3,90.8,93.7,95.9,96.9,99.5,97.6,...,92.9,91.6,92.6,93.4,91.9,89.0,89.3,90.6,0.07,5.6
6,Connecticut,43.0,41.7,40.1,42.8,44.6,44.1,41.1,40.2,37.7,...,35.1,34.4,34.8,35.2,36.5,34.3,34.1,37.6,-0.13,-5.4
7,Delaware,16.0,15.5,15.3,15.8,16.1,16.8,15.8,16.6,15.7,...,12.6,13.7,13.3,13.0,13.1,13.7,13.0,13.3,-0.17,-2.7
8,District of Columbia,4.3,4.1,4.2,3.9,4.0,3.9,3.2,3.4,3.1,...,3.1,2.7,2.8,3.0,3.0,2.8,2.6,2.9,-0.33,-1.4
9,Florida,240.2,239.0,242.4,246.7,258.1,261.4,259.7,257.4,239.0,...,233.0,227.6,227.8,234.1,238.6,240.2,239.5,243.2,0.01,3.0


In [23]:
engine = create_engine("sqlite:///data/db.sqlite")

meta = MetaData()
meta.drop_all(engine)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)